In [2]:
import os
import numpy as np
import torch
from tqdm import tqdm
import json
from dual_network_3f import Dual3DCNN6 as Dual
# from dual_network import Dual3DCNN6 as Dual
import torch
from torch.utils.data import Dataset
import SimpleITK as sitk
import numpy as np
import glob
from utilities import create_list_from_master_json, read_json_file, split_data
import re
import glob
import random
from torch.utils.data import Dataset, DataLoader
import numpy as np
import SimpleITK as sitk
import torch
from utilities import list_patient_folders, prepare_data_nrrd, split_data
from monai.transforms import Compose, LoadImaged, EnsureChannelFirstd, Spacingd, ScaleIntensityd, SpatialPadd, CenterSpatialCropd, ScaleIntensityRanged
from monai.data import CacheDataset, DataLoader, Dataset
from monai.transforms import LoadImaged
from monai.data.image_reader import ITKReader
from monai.data import SmartCacheDataset

In [16]:
from utilities import list_patient_folders, prepare_data_nrrd, split_data

# Specify the directory where the patient folders are located
data_path_NEW = '/home/shahpouriz/Data/DBP_newDATA/DBP/nrrd/test'
# data_path_OLD = '/home/shahpouriz/Data/DBP_oldDATA/nrrd/test'

import random
patient_list_NEW = list_patient_folders(data_path_NEW)
# Shuffle patient list if you want randomness
random.shuffle(patient_list_NEW)

# Define split sizes
total_patients = len(patient_list_NEW)
train_size = int(total_patients * 0.7)
val_size = int(total_patients * 0.20)
# The rest will be for the test set

# Split the patient list
train_patients = patient_list_NEW[:train_size]
val_patients = patient_list_NEW[train_size:train_size + val_size]
test_patients = patient_list_NEW[train_size + val_size:]

# Now you can prepare your data
# This step will depend on how your 'prepare_data_nrrd' function works
# You need to pass the right patient list to this function for each set
train_pct, train_rct, train_pos = prepare_data_nrrd(data_path_NEW, train_patients)
val_pct, val_rct, val_pos = prepare_data_nrrd(data_path_NEW, val_patients)
test_pct, test_rct, test_pos = prepare_data_nrrd(data_path_NEW, test_patients)

# Create dictionaries for each dataset
train_data = [{"plan": img, "repeat": tar, "pos": pos} for img, tar, pos in zip(train_pct, train_rct, train_pos)]
val_data = [{"plan": img, "repeat": tar, "pos": pos} for img, tar, pos in zip(val_pct, val_rct, val_pos)]
test_data = [{"plan": img, "repeat": tar, "pos": pos} for img, tar, pos in zip(test_pct, test_rct, test_pos)]


# Check the lengths of the sets
print("Number of training samples:", len(train_data))
print("Number of validation samples:", len(val_data))
print("Number of test samples:", len(test_data))

Number of training samples: 65
Number of validation samples: 67
Number of test samples: 43


In [15]:
list

['DBP_HN028',
 'DBP_HN025',
 'DBP_HN045',
 'DBP_HN010',
 'DBP_HN024',
 'DBP_HN042',
 'DBP_HN027',
 'DBP_HN004',
 'DBP_HN037',
 'DBP_HN009',
 'DBP_HN008',
 'DBP_HN036',
 'DBP_HN014',
 'DBP_HN011',
 'DBP_HN023',
 'DBP_HN012',
 'DBP_HN031',
 'DBP_HN044',
 'DBP_HN029',
 'DBP_HN020',
 'DBP_HN002',
 'DBP_OP013',
 'DBP_OP008',
 'DBP_OP017',
 'DBP_OP024',
 'DBP_OP004',
 'DBP_OP010',
 'DBP_OP020',
 'DBP_OP009',
 'DBP_OP027',
 'DBP_OP025',
 'DBP_OP014',
 'DBP_OP012',
 'DBP_OP001',
 'DBP_OP018',
 'DBP_OP026',
 'DBP_OP007',
 'DBP_OP006',
 'DBP_OP029',
 'DBP_OP011',
 'DBP_OP015',
 'DBP_OP022',
 'DBP_OP021',
 'DBP_OP023',
 'DBP_OP032',
 'DBP_OP028',
 'DBP_OP002']

In [11]:
test = [{"plan": img[0], "repeat": tar, "pos": pos} for img, tar, pos in zip(pct[:10], rct[:10], pos[:10])]
test

[{'plan': '/home/shahpouriz/Data/DBP_newDATA/DBP/nrrd/test/DBP_OP013/DBP_OP013_P1_planningCT.nrrd',
  'repeat': '/home/shahpouriz/Data/DBP_newDATA/DBP/nrrd/test/DBP_OP013/DBP_OP013_P1_repeatedCT5.nrrd',
  'pos': array([-0.71592,  0.21269,  1.25212], dtype=float32)},
 {'plan': '/home/shahpouriz/Data/DBP_newDATA/DBP/nrrd/test/DBP_OP013/DBP_OP013_P1_planningCT.nrrd',
  'repeat': '/home/shahpouriz/Data/DBP_newDATA/DBP/nrrd/test/DBP_OP013/DBP_OP013_P1_repeatedCT4.nrrd',
  'pos': array([ 0.63696, -0.06812, -0.55098], dtype=float32)},
 {'plan': '/home/shahpouriz/Data/DBP_newDATA/DBP/nrrd/test/DBP_OP013/DBP_OP013_P1_planningCT.nrrd',
  'repeat': '/home/shahpouriz/Data/DBP_newDATA/DBP/nrrd/test/DBP_OP013/DBP_OP013_P1_repeatedCT1.nrrd',
  'pos': array([ 0.04297, -0.09644,  0.06532], dtype=float32)},
 {'plan': '/home/shahpouriz/Data/DBP_newDATA/DBP/nrrd/test/DBP_OP013/DBP_OP013_P1_planningCT.nrrd',
  'repeat': '/home/shahpouriz/Data/DBP_newDATA/DBP/nrrd/test/DBP_OP013/DBP_OP013_P1_repeatedCT3.nrr

In [15]:
# Set parameters
starting_epoch = 0
# decay_epoch = 20
final_epoch = 6
# learning_rate = 0.0001
lambda_reg = 0.000001

# Condition for saving list
# save_list = False
best_mae = np.inf

exception_list = ['']


In [16]:
#### My method


from monai.transforms import Compose, LoadImaged, EnsureChannelFirstd, Spacingd, SpatialPadd, CenterSpatialCropd, ScaleIntensityRanged
from monai.data import CacheDataset, DataLoader, Dataset
from monai.transforms import LoadImaged
from monai.data.image_reader import ITKReader
batch_size = 2
dim = 128
size = (dim, dim, dim)
transforms = Compose([
        LoadImaged(keys=["plan", "repeat"], reader=ITKReader()),
        EnsureChannelFirstd(keys=["plan", "repeat"]),
        ScaleIntensityd(keys=["plan", "repeat"]),
        Spacingd(keys=["plan", "repeat"], pixdim=(3.0, 3.0, 3.0), mode='trilinear'),
        SpatialPadd(keys=["plan", "repeat"], spatial_size=size, mode='constant'),  # Ensure minimum size
        CenterSpatialCropd(keys=["plan", "repeat"], roi_size=size),  # Ensure uniform size
    ])


# train_ds = CacheDataset(data=train_data, transform=transforms, cache_rate=1.0, num_workers=2)
# train_loader = DataLoader(train_ds, batch_size=batch_size, shuffle=True, num_workers=2)

# val_ds = CacheDataset(data=val_data, transform=transforms, cache_rate=1.0, num_workers=2)
# val_loader = DataLoader(val_ds, batch_size=batch_size, shuffle=False, num_workers=2)

train_ds = SmartCacheDataset(data=train_data, transform=transforms, cache_rate=0.1, replace_rate=0.2, num_init_workers=2, num_replace_workers=2)
train_loader = DataLoader(train_ds, batch_size=batch_size, shuffle=True, num_workers=2)

val_ds = SmartCacheDataset(data=val_data, transform=transforms, cache_rate=0.1, replace_rate=0.2, num_init_workers=2, num_replace_workers=2)
val_loader = DataLoader(val_ds, batch_size=batch_size, shuffle=False, num_workers=2)




Loading dataset:   0%|          | 0/85 [00:00<?, ?it/s]

Loading dataset: 100%|██████████| 18/18 [00:00<00:00, 58.29it/s]


In [17]:
# from monai.transforms import Compose, LoadImaged, EnsureChannelFirstd, Spacingd, SpatialPadd, CenterSpatialCropd, ScaleIntensityRanged
# from monai.data import CacheDataset, DataLoader, Dataset
# import matplotlib.pyplot as plt

# # Assuming `transforms` is defined and `val_files` contains your validation files
# check_ds = Dataset(data=val_data, transform=transforms)
# check_loader = DataLoader(check_ds, batch_size=1)

# # Manually retrieve the first batch of data
# for check_data in check_loader:
#     break

# plan, repeat = (check_data["plan"][0][0], check_data["repeat"][0][0])
# print(f"image shape: {plan.shape}, target shape: {repeat.shape}")

# # plot the slice [:, :, n]
# n = 100

# plt.figure("check", (12, 6))
# plt.subplot(1, 2, 1)
# plt.title("image")
# plt.imshow(plan[:, :, n], origin="lower")
# plt.subplot(1, 2, 2)
# plt.title("repeat")
# plt.imshow(repeat[:, :, n])
# plt.show()

# lr_finder:

In [18]:
# import matplotlib.pyplot as plt
# import numpy as np
# import torch

# def find_lr(model, train_loader, optimizer, init_value=1e-8, final_value=10., beta=0.98):
#     num = len(train_loader)-1  # Total number of batches
#     mult = (final_value / init_value) ** (1/num)
#     lr = init_value
#     optimizer.param_groups[0]['lr'] = lr
#     avg_loss = 0.
#     best_loss = 0.
#     batch_num = 0
#     losses = []
#     log_lrs = []
#     for data in train_loader:
#         batch_num += 1
#         # Get the input and target from the batch
#         # This part needs to be adapted to your specific data loader structure
#         # Adjust these lines to match how your data is structured and ensure both inputs are provided
#         inputs_fixed = data["plan"].to(device)
#         inputs_moving = data["repeat"].to(device)
#         targets = data["pos"].to(device)  # Assuming 'pos' is the target
        
#         # Forward pass
#         optimizer.zero_grad()
#         # Make sure to pass both inputs to the model
#         outputs = model(inputs_fixed, inputs_moving)  # Adjusted to pass both inputs
#         # Assuming your model outputs in a format that matches targets, adjust if it outputs a tuple
#         loss = mae_loss(outputs, targets)

#         # Compute the smoothed loss
#         avg_loss = beta * avg_loss + (1-beta) * loss.item()
#         smoothed_loss = avg_loss / (1 - beta**batch_num)

#         # Stop if the loss is exploding
#         if batch_num > 1 and smoothed_loss > 4 * best_loss:
#             return log_lrs, losses

#         # Record the best loss
#         if smoothed_loss < best_loss or batch_num==1:
#             best_loss = smoothed_loss

#         # Store the values
#         losses.append(smoothed_loss)
#         log_lrs.append(np.log10(lr))

#         # Do the backward pass and optimize
#         loss.backward()
#         optimizer.step()

#         # Update the lr for the next step and store
#         lr *= mult
#         optimizer.param_groups[0]['lr'] = lr

#     return log_lrs, losses

# # Call the function
# model.train()
# log_lrs, losses = find_lr(model, train_loader, optimizer)

# # Plotting
# plt.figure(figsize=(6,4))
# plt.plot(log_lrs, losses)
# plt.xlabel("Learning Rate (log scale)")
# plt.ylabel("Loss")
# plt.show()


-----

In [19]:
# Build model
print('Initializing model...')
model = Dual(width=dim, height=dim, depth=dim)
device = torch.device("cuda:0")
model.to(device)

# Define loss
print('Defining loss...')
mae_loss = torch.nn.L1Loss()
# mae_loss = torch.nn.MSELoss()
from torch.optim.lr_scheduler import ReduceLROnPlateau
learning_rate = 1e-3  # Start with a learning rate
optimizer = torch.optim.Adam(model.parameters(), lr=learning_rate)

# Scheduler (optional, to reduce learning rate when a metric has stopped improving)
# Here, we use ReduceLROnPlateau which reduces learning rate when a metric stops improving
scheduler = ReduceLROnPlateau(optimizer, mode='min', factor=0.1, patience=10, verbose=True)


Initializing model...
Defining loss...


In [20]:
# Training loop

# Initialize best_mae before the training loop
best_mae = float('inf')
train_losses = []

for epoch in range(starting_epoch, final_epoch):
    model.train()  # Set model to training mode
    mae_list = []
    train_loss = []
    # Assuming mae_loss is defined and using the correct device
    for i, batch_data in enumerate(train_loader):
        pCT, rCT = batch_data["plan"].to(device), batch_data["repeat"].to(device)
        reg = batch_data["pos"].to(device)  # No need for clone().detach().requires_grad_(True) for targets
        
        # Forward pass
        outx, outy, outz = model(pCT, rCT)
        
        # Split reg into its components (assuming reg has shape [batch_size, 3])
        regx, regy, regz = reg[:, 0], reg[:, 1], reg[:, 2]

        # Calculate MAE for each component
        loss_x = mae_loss(outx, regx.unsqueeze(1))  # Add dimension to match output shape
        loss_y = mae_loss(outy, regy.unsqueeze(1))
        loss_z = mae_loss(outz, regz.unsqueeze(1))

        # Combine the losses (you could also weigh them differently)
        total_loss = (loss_x + loss_y + loss_z)/batch_size
        train_losses.append(total_loss.item())
        
        # Calculate average validation loss
        avg_train_loss = np.mean(train_losses)

        # Backpropagation
        optimizer.zero_grad()
        total_loss.backward()

        optimizer.step()
        
        # Logging (example for total_loss, adjust as needed)
        # print(f'Epoch: {epoch}/{final_epoch}, Batch: {i+1}/{len(train_loader)}, Loss: {total_loss.item()}')

    # Validation loop
    model.eval()
    val_losses = []
    with torch.no_grad():
        for batch_data in val_loader:
            pCT_val, rCT_val = batch_data["plan"].to(device), batch_data["repeat"].to(device)
            reg_val = batch_data["pos"].to(device)  # Ground truth coordinates
            
            # Model prediction
            outx_val, outy_val, outz_val = model(pCT_val, rCT_val)
       
            # Split reg_val into its components
            regx_val, regy_val, regz_val = reg_val[:, 0], reg_val[:, 1], reg_val[:, 2]
          
            # Calculate validation loss for each component
            loss_val_x = mae_loss(outx_val, regx_val.unsqueeze(1))
            loss_val_y = mae_loss(outy_val, regy_val.unsqueeze(1))
            loss_val_z = mae_loss(outz_val, regz_val.unsqueeze(1))
            
            # Combine the losses
            total_val_loss = (loss_val_x + loss_val_y + loss_val_z)/batch_size
            print(regx_val)
            val_losses.append(total_val_loss.item())
        
        # Calculate average validation loss
        avg_val_loss = np.mean(val_losses)
        # print(f'Epoch [{epoch+1}/{final_epoch}], Validation Loss: {avg_val_loss:.4f}')


        # Adjust learning rate
        scheduler.step(avg_val_loss)
        
        


        # Saving model and logging
        save_dir = '/home/shahpouriz/Data/DBP_Project/LOG'
        filename = f'loss_Model_3f_1p'
        loss_file = os.path.join(save_dir, f'{filename}.txt')

        if not os.path.isdir(save_dir):
            os.makedirs(save_dir)
        
        if avg_val_loss <= best_mae and epoch > 0:
            best_mae = avg_val_loss
            model_filename = f'{filename}_{epoch+1}.pt'  # Store the model filename
            torch.save(model.state_dict(), os.path.join(save_dir, model_filename))
            print(f'Model saved to {os.path.join(save_dir, model_filename)}')
        
        # Append current epoch's average loss and validation loss to the log file
        with open(loss_file, 'a') as f:
            f.write(f'Epoch: {epoch+1}/{final_epoch}, Loss: {avg_train_loss:.4f}, Val Loss: {avg_val_loss:.4f}\n')
            # f.write(f'Epoch: {epoch+1}/{final_epoch}, Loss: {avg_train_loss}, Val: {mean_val_loss}\n')


tensor([-1.1872,  1.0014], device='cuda:0')
tensor([-0.1480,  0.1944], device='cuda:0')
tensor([-0.8255,  1.2423], device='cuda:0')
tensor([ 0.1336, -0.0396], device='cuda:0')
tensor([-0.8695, -1.0548], device='cuda:0')
tensor([ 0.3498, -1.0046], device='cuda:0')
tensor([-1.1555,  0.4691], device='cuda:0')
tensor([-1.0963, -0.7831], device='cuda:0')
tensor([ 0.2657, -0.0995], device='cuda:0')
tensor([-1.1872,  1.0014], device='cuda:0')
tensor([-0.1480,  0.1944], device='cuda:0')
tensor([-0.8255,  1.2423], device='cuda:0')
tensor([ 0.1336, -0.0396], device='cuda:0')
tensor([-0.8695, -1.0548], device='cuda:0')
tensor([ 0.3498, -1.0046], device='cuda:0')
tensor([-1.1555,  0.4691], device='cuda:0')
tensor([-1.0963, -0.7831], device='cuda:0')
tensor([ 0.2657, -0.0995], device='cuda:0')
Model saved to /home/shahpouriz/Data/DBP_Project/LOG/loss_Model_3f_1p_2.pt


KeyboardInterrupt: 

In [ ]:

# # Initialize best_mae before the training loop
# best_mae = float('inf')
# train_losses = []

# # Training loop
# for epoch in range(starting_epoch, final_epoch):
#     model.train()  # Set model to training mode
#     mae_list = []
#     train_loss = []
#     # Assuming mae_loss is defined and using the correct device
#     for i, batch_data in enumerate(train_loader):
#         pCT, rCT = batch_data["plan"].to(device), batch_data["repeat"].to(device)
#         reg = batch_data["pos"].to(device)  # No need for clone().detach().requires_grad_(True) for targets
        
#         # Forward pass
#         outx, outy, outz = model(pCT, rCT)
        
#         # Split reg into its components (assuming reg has shape [batch_size, 3])
#         regx, regy, regz = reg[:, 0], reg[:, 1], reg[:, 2]

#         # Calculate MAE for each component
#         loss_x = mae_loss(outx, regx.unsqueeze(1))  # Add dimension to match output shape
#         loss_y = mae_loss(outy, regy.unsqueeze(1))
#         loss_z = mae_loss(outz, regz.unsqueeze(1))

#         # Combine the losses (you could also weigh them differently)
#         total_loss = (loss_x + loss_y + loss_z)/batch_size
#         train_losses.append(total_loss.item())
        
#         # Calculate average validation loss
#         avg_train_loss = np.mean(train_losses)
#         # Backpropagation
#         optimizer.zero_grad()
#         total_loss.backward()
#         # loss_y.backward()
#         # loss_z.backward()

#         optimizer.step()
        
#         # Logging (example for total_loss, adjust as needed)
#         print(f'Epoch: {epoch}/{final_epoch}, Batch: {i+1}/{len(train_loader)}, Loss: {total_loss.item()}')

#     # Validation loop
#     model.eval()
#     val_losses = []
#     with torch.no_grad():
#         for batch_data in val_loader:
#             pCT_val, rCT_val = batch_data["plan"].to(device), batch_data["repeat"].to(device)
#             reg_val = batch_data["pos"].to(device)  # Ground truth coordinates
            
#             # Model prediction
#             outx_val, outy_val, outz_val = model(pCT_val, rCT_val)
#             print(outx_val, outy_val, outz_val)
#             # Split reg_val into its components
#             regx_val, regy_val, regz_val = reg_val[:, 0], reg_val[:, 1], reg_val[:, 2]
#             print(regx_val, regy_val, regz_val)
#             # Calculate validation loss for each component
#             loss_val_x = mae_loss(outx_val, regx_val.unsqueeze(1))
#             loss_val_y = mae_loss(outy_val, regy_val.unsqueeze(1))
#             loss_val_z = mae_loss(outz_val, regz_val.unsqueeze(1))
            
#             # Combine the losses
#             total_val_loss = (loss_val_x + loss_val_y + loss_val_z)/batch_size
            
#             val_losses.append(total_val_loss.item())
        
#         # Calculate average validation loss
#         avg_val_loss = np.mean(val_losses)
#         print(f'Epoch [{epoch+1}/{final_epoch}], Validation Loss: {avg_val_loss:.4f}')


#         # Adjust learning rate
#         lr_scheduler.step()
        
        


#         # Saving model and logging
#         save_dir = '/home/shahpouriz/Data/DBP_Project/LOG'
#         filename = f'loss_Model_3f'
#         loss_file = os.path.join(save_dir, f'{filename}.txt')

#         if not os.path.isdir(save_dir):
#             os.makedirs(save_dir)
        
#         if avg_val_loss <= best_mae and epoch > 0:
#             best_mae = avg_val_loss
#             model_filename = f'{filename}_{epoch+1}.pt'  # Store the model filename
#             torch.save(model.state_dict(), os.path.join(save_dir, model_filename))
#             print(f'Model saved to {os.path.join(save_dir, model_filename)}')
        
#         # Append current epoch's average loss and validation loss to the log file
#         with open(loss_file, 'a') as f:
#             f.write(f'Epoch: {epoch+1}/{final_epoch}, Loss: {avg_train_loss:.4f}, Val Loss: {avg_val_loss:.4f}\n')
#             # f.write(f'Epoch: {epoch+1}/{final_epoch}, Loss: {avg_train_loss}, Val: {mean_val_loss}\n')


In [ ]:
# Training loop
for epoch in range(starting_epoch, final_epoch):
    
    # model.train()  # Set model to training mode
    mae_list = []
    train_loss = []
    for i, batch_data in enumerate(train_loader):  # Use enumerate to get the batch index
        pCT, rCT = batch_data["plan"].to(device), batch_data["repeat"].to(device)
        reg = batch_data["pos"].clone().detach().requires_grad_(True).to(device)  # If gradients are required for 'reg'
        print("*****************************************")
        print(reg)

        # Forward pass
        output = model(pCT, rCT)
        print(output)
        loss_output = mae_loss(output, reg)
        print(loss_output)


        # L1 Regularization
        l1_reg = torch.tensor(0., requires_grad=True).to(device)
        for name, param in model.named_parameters():
            l1_reg = l1_reg + torch.norm(param, 1)
        loss_output += lambda_reg * l1_reg
        print(loss_output)
        
        # Backpropagation
        optimizer.zero_grad()
        loss_output.backward()  
        optimizer.step()
        
        # Logging
        mae_list.append(loss_output.item())
        mean_mae = np.mean(mae_list)
        # Corrected to print the current batch number
        print(f'Epoch: {epoch}/{final_epoch}, Batch: {i+1}/{len(train_loader)}, Loss_avg: {mean_mae}')

    # Validation loop
    # model.eval()
    val_loss = []
    with torch.no_grad():
        for batch_data in val_loader:
            pCT_val, rCT_val = batch_data["plan"].to(device), batch_data["repeat"].to(device)
            reg_val = batch_data["pos"].clone().detach().requires_grad_(True).to(device)  # If gradients are required for 'reg'

            output_val = model(pCT_val, rCT_val)
            loss_output_val = mae_loss(output_val, reg_val)

            val_loss.append(loss_output_val.item())

            mean_val_loss = np.mean(val_loss)
            print(f'Epoch [{epoch+1}/{final_epoch}], Validation Loss: {mean_val_loss:.4f}')

    # Adjust learning rate
    lr_scheduler.step()
    
    save_dir = '/home/shahpouriz/Data/DBP_Project/LOG'
    filename = f'loss_simple_Model_{dim}'
    loss_file = fr'/home/shahpouriz/Data/DBP_Project/LOG/{filename}.txt'

    if not os.path.isdir(save_dir):
        os.makedirs(save_dir)
    
    current_valid_mae = val_loss[-1]    
    print(f'Epoch: {epoch+1}, Current Valid MAE: {current_valid_mae}, Best MAE: {best_mae}')
    if current_valid_mae <= best_mae and epoch > 0:
        best_mae = current_valid_mae
        model_filename = f'{filename}_{epoch+1}.pt'
        model_path = f'{save_dir}/{model_filename}'
        torch.save(model.state_dict(), model_path)
        print(f'Model saved: {model_path}')
    else:
        pass


*****************************************
tensor([[-0.8186, -0.2103, -1.5246],
        [-1.0367, -0.3661,  0.6014]], device='cuda:0',
       grad_fn=<ToCopyBackward0>)
(metatensor([[-0.2432],
        [-0.2429]], device='cuda:0', grad_fn=<AliasBackward0>), metatensor([[-0.0306],
        [-0.0305]], device='cuda:0', grad_fn=<AliasBackward0>), metatensor([[0.0992],
        [0.0991]], device='cuda:0', grad_fn=<AliasBackward0>))


AttributeError: 'tuple' object has no attribute 'size'

In [ ]:
# count = 0
# for pct_tensor, rct_tensor, pos in train_dataset:
#     print("pCT Tensor:", pct_tensor.shape)
#     print("rCT Tensor:", rct_tensor.shape)
#     print("Position:", pos)
#     count += 1
#     if count == 1:
#         break



In [ ]:

# # Build model
# print('Initializing model...')
# model = Dual(width=512, height=512, depth=512)
# device = torch.device(f"cuda:{device_num}" if torch.cuda.is_available() else "cpu")
# model.to(device)

# # Define loss
# print('Defining loss...')
# mae_loss = torch.nn.L1Loss()
# mse_loss = torch.nn.MSELoss()

# # Define optimizer
# print('Defining optimizer...')
# optimizer = torch.optim.Adam(model.parameters(), lr=learning_rate, betas=(0.5, 0.999))

# # Define scheduler
# print('Defining scheduler...')
# lr_lambda = DecayLR(epochs=final_epoch, offset=0, decay_epochs=decay_epoch).step
# lr_scheduler = torch.optim.lr_scheduler.LambdaLR(optimizer, lr_lambda=lr_lambda)


In [ ]:
# # Initialize model
# model = Dual(width=512, height=512, depth=512)
# device = torch.device("cuda" if torch.cuda.is_available() else "cpu")
# model.to(device)

# # Define loss
# mae_loss = torch.nn.L1Loss()
# mse_loss = torch.nn.MSELoss()

# # Define optimizer
# optimizer = torch.optim.Adam(model.parameters(), lr=learning_rate, betas=(0.5, 0.999))

# # Define scheduler
# lr_lambda = DecayLR(epochs=final_epoch, offset=0, decay_epochs=decay_epoch).step
# lr_scheduler = torch.optim.lr_scheduler.LambdaLR(optimizer, lr_lambda=lr_lambda)

# # Training loop
# for epoch in range(starting_epoch, final_epoch):
#     mae_list = []
#     model.train()  # Set model to training mode
#     for i, (pCT, rCT, reg) in enumerate(train_loader):
#         pCT = pCT.to(device)
#         rCT = rCT.to(device)
#         reg = reg = reg[0].unsqueeze(1).to(device)  # Access the element of the list and move it to the device


        
#         # Forward pass
#         output = model(pCT, rCT)
#         output = output.to(torch.float32)
#         reg = reg.to(torch.float32)
#         # Calculate loss
#         loss_output = mse_loss(output, reg)

            
#         # L1 Regularization
#         l1_reg = torch.tensor(0., requires_grad=True).to(device)
#         for name, param in model.named_parameters():
#             l1_reg = l1_reg + torch.norm(param, 1)
#         loss_output += lambda_reg * l1_reg
        
#         # Backpropagation
#         optimizer.zero_grad()
#         loss_output.backward()
#         optimizer.step()
        
#         # Logging
#         mae_list.append(loss_output.item())
#         mean_mae = np.mean(mae_list)
#         print(f'Epoch: {epoch}/{final_epoch}, Batch: {i}/{len(train_loader)}, Loss_avg: {mean_mae}')
    
#     # Validation loop
#     model.eval()  # Set model to evaluation mode
#     val_loss = []
#     with torch.no_grad():
#         for j, (pCT_val, rCT_val, reg_val) in enumerate(val_loader):
#             pCT_val = pCT_val.to(device)
#             rCT_val = rCT_val.to(device)
#             reg_val = reg_val.to(device)
            
#             output_val = model(pCT_val, rCT_val)
#             loss_output_val = mae_loss(output_val, reg_val)
            
#             val_loss.append(loss_output_val.item())
#             mean_val_loss = np.mean(val_loss)
#             print(f'Epoch: {epoch}/{final_epoch}, Batch: {j}/{len(val_loader)}, Loss_avg: {mean_val_loss}')
    
#     # Adjust learning rate
#     lr_scheduler.step(mean_val_loss)
    
#     save_dir = '/home/shahpouriz/Data/DBP_Project/LOG'
#     fname_comment = 'test'
#     loss_file = fr'/home/shahpouriz/Data/DBP_Project/LOG/loss_dose_json_simpleModel_{fname_comment}.txt'

#     # Save model
#     if not os.path.isdir(save_dir):
#         os.makedirs(save_dir)
    
#     current_valid_mae = val_loss[-1]    
#     if current_valid_mae <= best_mae and epoch > 0:
#         best_mae = current_valid_mae
#         torch.save(model.state_dict(),f'{save_dir}/model_weights_dose_{epoch+1}_{fname_comment}.pt')
#     with open(loss_file, 'a') as f: #a-append
#         f.write(f'Epoch: {epoch+1}/{final_epoch}, Loss: {mean_mae}, Val: {mean_val_loss}\n')
